<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/You_Tube_video_transcript_Summarizer_using_whisper_and_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU openai langchain tiktoken pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00


In [4]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.1 MB/s eta 0:00:00


In [15]:
import os
import openai
from getpass import getpass
#
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
#
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
import textwrap
#
from pytube import YouTube
import whisper

## Set up Openai Key

In [3]:
os.environ["OPENAI_API_KEY"] = getpass()

··········


## Instantiate models

In [6]:
def load_models():
  model = whisper.load_model("large")
  return model

In [7]:
model = load_models()

100%|██████████████████████████████████████| 2.87G/2.87G [00:21<00:00, 141MiB/s]


In [8]:
import tiktoken
tokenizer = tiktoken.get_encoding('cl100k_base')
#
def tiktoken_len(text):
  tokens = tokenizer.encode(text,disallowed_special=())
  return len(tokens)

In [13]:
def summarize_yt(url,model):
  yt = YouTube(url)
  audio = yt.streams.filter(only_audio=True).first().download()
  print("Audio downloaded")

  transcript = model.transcribe(audio,fp16=False)['text']
  print(transcript)

  text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,
                                                 chunk_overlap=20,
                                                 length_function = tiktoken_len,
                                                 separators = ['\n\n','\n','  ',' '])
  texts = text_splitter.split_text(transcript)
  docs = [Document(page_content=t) for t in texts]
  print("Text split")
  return docs

In [21]:
def summarize(docs):
  #
  prompt_template = """You are an expert at making strong factual summarizations.
  Extract the key facts out of the text provided enclosed within 3 bacticks. Don't include opinions.
  Give each fact a number and keep them short sentences. :\n\n"
  ```{text}```"""
  PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
  #
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
  #
  chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
  #
  response = chain.run(docs)
  return response



## Transcribe the Youtube link provided

In [16]:
%%time
url = "https://youtu.be/I2ZK3ngNvvI"
docs = summarize_yt(url,model)

Audio downloaded
 You're one of the greatest teachers of machine learning, AI ever, from CS231N to today. What advice would you give to beginners interested in getting into machine learning? Beginners are often focused on like what to do, and I think the focus should be more like how much you do. So I am kind of like believer on a high level in this 10,000 hours kind of concept where just kind of have to just pick the things where you can spend time and you care about and you're interested in. You literally have to put in 10,000 hours of work. It doesn't even matter as much like where you put it and you'll iterate and you'll improve and you'll waste some time. I don't know if there's a better way. You need to put in 10,000 hours, but I think it's actually really nice because I feel like there's some sense of determinism about being an expert at a thing. If you spend 10,000 hours, you can literally pick an arbitrary thing. And I think if you spend 10,000 hours of deliberate effort and w

## Response generated by Davinci

In [17]:
%%time
response = summarize(docs)
print(response)

1. The advice given to beginners interested in machine learning is to focus on the quantity of hours spent on learning and practicing.
2. Putting in 10,000 hours of deliberate effort and work is believed to make someone an expert in a particular field.
3. It is recommended to compare one's progress to their past self rather than comparing oneself to others.
4. Beginners often get paralyzed by the choices and worry about making mistakes, but mistakes and wasted time can lead to learning and growth.
5. The speaker is not particularly fond of teaching, but enjoys being helpful and appreciated by others.
6. Creating educational materials and lectures requires a lot of work, iteration, and thought.
7. The speaker prepares for lectures by doing multiple takes and selecting the better ones, and it takes around 10 hours to create one hour of content.
8. Teaching helps strengthen the speaker's understanding and reveals gaps in knowledge.
9. The speaker finds it fascinating to see the actual res

## Response Generated by GPT3

In [22]:
%%time
response = summarize(docs)
print(response)

1. Beginners in machine learning should focus on the quantity of hours they spend practicing and learning.
2. Putting in 10,000 hours of deliberate effort and work can make someone an expert in a particular field.
3. It is important to compare one's progress to their past self rather than comparing themselves to others.
4. Beginners should not get paralyzed by the choices and mistakes they make, as they can learn and grow from them.
5. Teaching can be challenging and time-consuming, requiring a lot of iteration and thought to create good educational materials.
6. Teaching helps strengthen one's understanding of the subject matter.
7. Explaining concepts to others can reveal gaps in one's knowledge.
8. Building and running code is a powerful way to learn and understand machine learning concepts.
CPU times: user 110 ms, sys: 6.96 ms, total: 117 ms
Wall time: 5.55 s


## Response generated by GP4

In [20]:
%%time
response = summarize(docs)
print(response)

1. The speaker believes in the concept of spending 10,000 hours on a task to become an expert at it.
2. The speaker advises beginners in machine learning to focus on the quantity of hours spent learning rather than what specifically to learn.
3. The speaker suggests forming a daily habit to maximize the likelihood of reaching 10,000 hours of practice.
4. The speaker discourages comparing oneself to others, instead suggesting to compare oneself to their past self to track progress.
5. The speaker acknowledges that mistakes and wasted time are part of the learning process and contribute to growth.
6. The speaker does not particularly love teaching, but does it because people appreciate it and it makes them happy.
7. The speaker finds creating educational materials to be hard work and not always enjoyable.
8. The speaker typically takes multiple takes when creating a lecture and it takes approximately 10 hours to create one hour of content.
9. The speaker finds that teaching strengthens t